In [1]:
import pandas as pd
import random

from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
import torch

/Users/anatolii/Desktop/movies_subtitles/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# device = "mps" if torch.mps.is_available() else "cpu"
device = "cpu"
device 

'cpu'

## Подготовка данных

In [3]:
df_read = pd.read_csv(r'medium_articles.csv')


In [4]:
df_read.head()

,title,text,url,authors,timestamp,tags
0,Mental Note Vol. 24,Photo by Josh Riemer on Unsplash\n\nMerry Chri...,https://medium.com/invisible-illness/mental-no...,['Ryan Fan'],2020-12-26 03:38:10.479000+00:00,"['Mental Health', 'Health', 'Psychology', 'Sci..."
1,Your Brain On Coronavirus,Your Brain On Coronavirus\n\nA guide to the cu...,https://medium.com/age-of-awareness/how-the-pa...,['Simon Spichak'],2020-09-23 22:10:17.126000+00:00,"['Mental Health', 'Coronavirus', 'Science', 'P..."
2,Mind Your Nose,Mind Your Nose\n\nHow smell training can chang...,https://medium.com/neodotlife/mind-your-nose-f...,[],2020-10-10 20:17:37.132000+00:00,"['Biotechnology', 'Neuroscience', 'Brain', 'We..."
3,The 4 Purposes of Dreams,Passionate about the synergy between science a...,https://medium.com/science-for-real/the-4-purp...,['Eshan Samaranayake'],2020-12-21 16:05:19.524000+00:00,"['Health', 'Neuroscience', 'Mental Health', 'P..."
4,Surviving a Rod Through the Head,"You’ve heard of him, haven’t you? Phineas Gage...",https://medium.com/live-your-life-on-purpose/s...,['Rishav Sinha'],2020-02-26 00:01:01.576000+00:00,"['Brain', 'Health', 'Development', 'Psychology..."


In [5]:
df_read.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192368 entries, 0 to 192367
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   title      192363 non-null  object
 1   text       192368 non-null  object
 2   url        192368 non-null  object
 3   authors    192368 non-null  object
 4   timestamp  192366 non-null  object
 5   tags       192368 non-null  object
dtypes: object(6)
memory usage: 8.8+ MB


In [6]:
df = df_read.dropna(subset=['text'])

In [7]:
print(f'Len movies subtitles: {len(df)}')
df.head()

Len movies subtitles: 192368


,title,text,url,authors,timestamp,tags
0,Mental Note Vol. 24,Photo by Josh Riemer on Unsplash\n\nMerry Chri...,https://medium.com/invisible-illness/mental-no...,['Ryan Fan'],2020-12-26 03:38:10.479000+00:00,"['Mental Health', 'Health', 'Psychology', 'Sci..."
1,Your Brain On Coronavirus,Your Brain On Coronavirus\n\nA guide to the cu...,https://medium.com/age-of-awareness/how-the-pa...,['Simon Spichak'],2020-09-23 22:10:17.126000+00:00,"['Mental Health', 'Coronavirus', 'Science', 'P..."
2,Mind Your Nose,Mind Your Nose\n\nHow smell training can chang...,https://medium.com/neodotlife/mind-your-nose-f...,[],2020-10-10 20:17:37.132000+00:00,"['Biotechnology', 'Neuroscience', 'Brain', 'We..."
3,The 4 Purposes of Dreams,Passionate about the synergy between science a...,https://medium.com/science-for-real/the-4-purp...,['Eshan Samaranayake'],2020-12-21 16:05:19.524000+00:00,"['Health', 'Neuroscience', 'Mental Health', 'P..."
4,Surviving a Rod Through the Head,"You’ve heard of him, haven’t you? Phineas Gage...",https://medium.com/live-your-life-on-purpose/s...,['Rishav Sinha'],2020-02-26 00:01:01.576000+00:00,"['Brain', 'Health', 'Development', 'Psychology..."


In [8]:
val_ind = set(random.sample(range(df.shape[0]), 1000))

train = df[~df.index.isin(val_ind)].iloc[:10000]['text'].tolist()
valid = df[df.index.isin(val_ind)]['text'].tolist()

print(f'Training set size: {len(train)}')
print(f'Validation set size: {len(valid)}')

Training set size: 10000
Validation set size: 1000


## Инициализация токенизатора и трансформерной модели

In [9]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [10]:
input_text = "Hello, world!"
inputs = tokenizer(input_text, return_tensors='pt').to(device)
outputs = model(**inputs)
print(outputs)

CausalLMOutputWithCrossAttentions(loss=None, logits=tensor([[[ -35.2361,  -35.3264,  -38.9752,  ...,  -44.4643,  -43.9973,
           -36.4578],
         [-112.6172, -114.5833, -116.5725,  ..., -119.0128, -118.8060,
          -111.6918],
         [ -79.2378,  -81.9410,  -87.2414,  ...,  -93.5786,  -90.7576,
           -85.6705],
         [-118.9063, -119.5878, -121.2070,  ..., -130.6018, -128.5587,
          -116.1679]]], grad_fn=<UnsafeViewBackward0>), past_key_values=((tensor([[[[-1.2526e+00,  2.3200e+00,  1.7218e-01,  ..., -1.0076e+00,
           -1.8970e-01,  1.3219e+00],
          [-1.6482e+00,  3.0222e+00,  1.2789e+00,  ..., -9.0779e-01,
           -1.7395e+00,  2.4237e+00],
          [-2.7801e+00,  3.8824e+00,  1.4420e+00,  ..., -2.3010e-01,
           -2.8068e+00,  1.4485e+00],
          [-2.0379e+00,  2.7107e+00,  1.9940e+00,  ..., -1.4642e+00,
           -1.9678e+00,  2.2460e+00]],

         [[-3.9746e-01, -4.7357e-01, -8.4727e-01,  ...,  3.0718e-01,
            2.2743e+00,  

## Тест готового трансформера "из коробки"

In [23]:
def generate_text(model, prompt, max_length=100, temperature=1.0, top_p=0.9, no_repeat_ngram_size=1):
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to('cpu')
    model.to('cpu')
    output = model.generate(
        input_ids=input_ids,
        max_length=max_length,
        temperature=temperature,
        pad_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=no_repeat_ngram_size,
        num_return_sequences=1,
    )
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
     
    return generated_text


prompt = "The adventure "
generated_text = generate_text(model, prompt, max_length=100, temperature=1.0, no_repeat_ngram_size=1)

print("Generated Text:")
print(generated_text)

Generated Text:
The adventure �I was a young man who had neverBuyableInstoreAndOnline.
” I have always wondered about the name, ‡What is it with us that we are not born of one?…' — The boy to his mother on my childhood and in-cage life for me! (A few years later) It seems he has changed but still there can be no doubt as yet you were an adventurous kid from your family will love everything when they see him). But what does this


## Дообучение с новым датасетом и его тестирование

In [11]:
class SubtitlesDataset(torch.utils.data.Dataset):
    def __init__(self, texts, tokenizer, max_length=128):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        return self.tokenizer(self.texts[idx], truncation=True, padding="max_length", max_length=self.max_length, return_tensors="pt")

train_dataset = SubtitlesDataset(train, tokenizer)
valid_dataset = SubtitlesDataset(valid, tokenizer)

In [13]:
%set_env PYTORCH_ENABLE_MPS_FALLBACK=1

env: PYTORCH_ENABLE_MPS_FALLBACK=1


In [14]:
training_args = TrainingArguments(
    output_dir='./results',  
    overwrite_output_dir=True,
    num_train_epochs=10,  
    per_device_train_batch_size=5,  
    save_steps=500,
    save_total_limit=1,
    prediction_loss_only=True,
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

In [15]:
trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
500,3.498500
1000,3.477000
1500,3.493900
2000,3.442000
2500,3.406300
3000,3.470500
3500,3.502400
4000,3.498000
4500,3.525300
5000,3.603400


TrainOutput(global_step=20000, training_loss=3.783190087890625, metrics={'train_runtime': 7058.0165, 'train_samples_per_second': 14.168, 'train_steps_per_second': 2.834, 'total_flos': 6532300800000000.0, 'train_loss': 3.783190087890625, 'epoch': 10.0})

In [16]:
model_path = './results/checkpoint-20000'
finetune_model = GPT2LMHeadModel.from_pretrained(model_path).to(device)
finetune_model.resize_token_embeddings(len(tokenizer))


Embedding(50258, 768)

In [20]:
prompt = "On the earth  "

generated_text = generate_text(finetune_model, prompt, max_length=100, temperature=1.0, top_p=0.9, no_repeat_ngram_size=1)

print("Generated Text:")
print(generated_text)

The following generation flags are not valid and may be ignored: ['top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Generated Text:
On the earth  𝐉
“The world is a great machine. I can tell you that it has been built by me, and my work on this project will be done through your help in our way of understanding its life to us! — Elon Musk (Photo: SpaceX) ‘I have worked for many years with some people who are not very good at science or technology but they know how much we do here.) The problem was when there were no other ways around as
